In [1]:
import os
import conda
import pandas as pd
import numpy as np
import json
from scipy.optimize import curve_fit
# from shapely.geometry import Polygon as Poly

pd.options.display.max_columns = 250

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.colors as colors

import geopandas
from geopandas.tools import sjoin
import geoplot as gplt
import geoplot.crs as gcrs

import folium
from folium.plugins import MousePosition, Search, FastMarkerCluster

import branca
import branca.colormap as cm

In [7]:
from folium.plugins import MousePosition, Search
import branca
import branca.colormap as cm

poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
poly['HUC_8'] = poly['HUC_8'].astype(str)
HUC8_json = poly.to_crs(epsg='4326').to_json()

AntrophogenicPBalance_df = pd.read_csv('DatabasesClean/AntrophogenicPBalance.csv', converters={'HUC_8': lambda x: str(x)}).fillna(np.nan)
AntrophogenicPBalance_df_GEOjson = poly.merge(AntrophogenicPBalance_df, on='HUC_8', how = 'left').fillna(np.nan)

m_AntrophogenicPBalance = folium.Map(location=[48, -102], tiles="openstreetmap", zoom_start=6)

# bins = list(AntrophogenicPBalance_df_GEOjson['P_TES'].quantile([0, 0.1, 0.2, 0.3,0.5,0.6,0.7,0.8,0.9, 1]))
# colormap = branca.colormap.linear.YlGnBu_09.scale(0, 30)

# colormap = branca.colormap.linear.YlOrRd_09.scale(0, 8500)
# colormap = colormap.to_step(index=[0, 1000, 3000, 5000, 8500])
# colormap.caption = 'Incidents of Crime in Victoria (year ending June 2018)'
# colormap.add_to(world_map)

# colormap = branca.colormap.linear.YlGnBu_09.scale(
#     AntrophogenicPBalance_df_GEOjson['P_TES'].min(),
#     AntrophogenicPBalance_df_GEOjson['P_TES'].max())

# colormap = cm.LinearColormap(colors=['red','blue'], index=[AntrophogenicPBalance_df_GEOjson['P_TES'].min(),0],
#                              vmin=AntrophogenicPBalance_df_GEOjson['P_TES'].min(),vmax=AntrophogenicPBalance_df_GEOjson['P_TES'].max())

colormap = cm.StepColormap(colors=['#AD1E2E','#2A67A6'], index=[AntrophogenicPBalance_df_GEOjson['P_TES'].min(),0],
                             vmin=AntrophogenicPBalance_df_GEOjson['P_TES'].min(),vmax=AntrophogenicPBalance_df_GEOjson['P_TES'].max())

# colormap.to_step(index=[-200, 0, 1200])


# Anthropogenic_P_Uptake_layer = folium.Choropleth(
#     geo_data=HUC8_json,
#     name='Anthropogenic P Uptake',
#     data=AnthropogenicHUC_PUptake_df_GEOjson,
#     columns=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
#     key_on='feature.properties.HUC_8',
# #     bins = bins,
#     fill_color=colorscale,
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     nan_fill_color='grey',
#     nan_fill_opacity=0.7,
#     legend_name=r"Anthropogenic P uptake (kg/year)",
#     highlight=True,
# ).add_to(m_Anthropogenic_PUptake)

# folium.GeoJson(
#     AnthropogenicHUC_PUptake_df_GEOjson.to_json(),
#     style_function=lambda feature: {
#         'fillOpacity': 0,
#         'color': 'black',
#         'weight': 0.1,
#     },
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
#         aliases=['HUC8 watershed', 'Anthropogenic P uptake (kg/year)'], 
#         localize=True)
#     ).add_to(m_Anthropogenic_PUptake)



P_TES_dict = AntrophogenicPBalance_df_GEOjson.set_index('HUC_8')['P_TES']

AntrophogenicPBalance_layer = folium.GeoJson(
    AntrophogenicPBalance_df_GEOjson.to_json(),
    name='P TES',
    style_function=lambda feature: {
        'fillColor': 'grey' if np.isnan(P_TES_dict[feature['properties']['HUC_8']]) == True else colormap(P_TES_dict[feature['properties']['HUC_8']]),
        'fillOpacity': 0.9,
        'color': 'black',
        'weight': 0.9,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['HUC_8', 'P_TES'],
        aliases=['HUC8 watershed', 'P TES'], 
        localize=True)
    ).add_to(m_AntrophogenicPBalance)

# colormap.caption = 'P TES'
# colormap.add_to(m_AntrophogenicPBalance)

folium.LayerControl().add_to(m_AntrophogenicPBalance)

formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='bottomright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m_AntrophogenicPBalance)

HUC8search = Search(
    layer=AntrophogenicPBalance_layer,
    geom_type='Polygon',
    placeholder='Search for a HUC8 watershed',
    collapsed=False,
    search_label='HUC_8',
    weight=3
).add_to(m_AntrophogenicPBalance)

#Customized legend
legend_html =   '''  
                <div id='legend', 
                style="position: fixed; 
                            bottom: 50px; right: 50px; width: 250px; height: 175px; 
                            padding: 10px 10px;
                            border:2px solid grey; z-index:9999; font-size:16px;
                            background:#ffffff;
                            ">
                    <p><strong>Balance of anthropogenic phosphorus releases<br>(TES metric)</strong></p>
                    <p>
                    <span style='color:#AD1E2E'>▉</span> Unbalanced<br>
                    <span style='color:#2A67A6'>▉</span> Balanced<br>
                    <span style='color:#848889'>▉</span> No data available<br>
                    </p>
                </div>'
                ''' 
m_AntrophogenicPBalance.get_root().html.add_child(folium.Element(legend_html))

m_AntrophogenicPBalance.save('m_AntrophogenicPBalance_colormap.html')